In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')


In [ ]:
file_path = 'C:/Users/Admin/Desktop/DROUGHT-FORECASTING-IN-KENYA-USING-MACHINE-LEARNING/MLFile2.xls'  # Replace with actual path
sheet_names = pd.ExcelFile(file_path).sheet_names
sheet_dict = {sheet: pd.read_excel(file_path, sheet_name=sheet) for sheet in sheet_names}
print (sheet_dict)



In [ ]:
filtered_sheet_dict = {}
correlation_dict = {} 

for station, df in sheet_dict.items():
    print(f"\nProcessing station: {station}")
    print(f"Original columns: {df.columns.tolist()}")
    
    df_copy = df.copy()
    
    df_copy.dropna(inplace=True)
    

    if df_copy.empty:
        print(f"No data left for {station} after dropping missing values. Skipping...")
        continue
    

    year_col = "YEAR"
    rainfall_col = station  
    
    print(f"Year column: {year_col}")
    print(f"Rainfall column: {rainfall_col}")
    
    sst_cols = [col for col in df_copy.columns if col not in [year_col, rainfall_col]]
    print(f"SST columns: {sst_cols}")
    
    print(f"\nCorrelation results for {station}:")
    
    retained_cols = []
    station_correlations = {}  # Store correlations for this station
    
    for col in sst_cols:
        try:
            corr, p_value = stats.pearsonr(df_copy[col], df_copy[rainfall_col])
            print(f"{col}: r={corr:.2f}, p={p_value:.3f}")
            station_correlations[col] = {'correlation': corr, 'p_value': p_value}  # Save correlation stats
            
            if p_value <= 0.05:
                retained_cols.append(col)
            else:
                print(f"Dropped {col} from {station} (p={p_value:.3f})")
        except Exception as e:
            print(f"Error processing column {col}: {str(e)}")
    
    correlation_dict[station] = station_correlations
    
    print(f"Retained {len(retained_cols)} predictors out of {len(sst_cols)} original variables")
    
    if not retained_cols:
        print(f"No significant predictors found for {station}. Skipping this station...")
        continue
    
    # Keep only YEAR, the rainfall column, and significant SST predictors
    filtered_df = df_copy[[year_col, rainfall_col] + retained_cols]
    filtered_sheet_dict[station] = filtered_df
    
    print(f"Final data for {station}:")
    display(filtered_df.head())
    print(f"Shape: {filtered_df.shape} (rows, columns)")
    print(f"Columns: {filtered_df.columns.tolist()}")
    print("-" * 60)

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# List of known El Niño years
el_nino_years = [1997, 2013, 2019, 2023]

# Create model combinations
model_configs = [
    {"Station": station_name, "Model": model_type} 
    for station_name in filtered_sheet_dict.keys() 
    for model_type in ["Random Forest", "Gradient Boosting", "SVR", "Neural Network"]
]
model_df = pd.DataFrame(model_configs)

# Container for results
results_dict = {}

# Function to add additional features that help with extreme values
def engineer_features(df):
    # Copy to avoid modifying original
    df_new = df.copy()
    
    # Calculate rolling averages if enough data is available
    if len(df) > 3:
        # If YEAR is in index, reset it
        if 'YEAR' in df_new.index.names:
            df_new = df_new.reset_index()
        
        # Sort by year for proper rolling calculations
        df_new = df_new.sort_values('YEAR')
        
        # Identify the rainfall column (assuming it's the second column)
        rainfall_col = df_new.columns[1]
        
        # Calculate rolling stats - using previous years' data
        # These features help the model recognize patterns leading to extreme values
        df_new['rolling_mean_3yr'] = df_new[rainfall_col].rolling(window=3, min_periods=1).mean().shift(1)
        df_new['rolling_std_3yr'] = df_new[rainfall_col].rolling(window=3, min_periods=1).std().shift(1)
        df_new['rolling_min_3yr'] = df_new[rainfall_col].rolling(window=3, min_periods=1).min().shift(1)
        df_new['rolling_max_3yr'] = df_new[rainfall_col].rolling(window=3, min_periods=1).max().shift(1)
        
        # Fill NA values with appropriate statistics
        df_new['rolling_mean_3yr'] = df_new['rolling_mean_3yr'].fillna(df_new[rainfall_col].mean())
        df_new['rolling_std_3yr'] = df_new['rolling_std_3yr'].fillna(df_new[rainfall_col].std())
        df_new['rolling_min_3yr'] = df_new['rolling_min_3yr'].fillna(df_new[rainfall_col].min())
        df_new['rolling_max_3yr'] = df_new['rolling_max_3yr'].fillna(df_new[rainfall_col].max())
    
    return df_new

# Loop through each model configuration
for _, row in model_df.iterrows():
    station = row["Station"]
    model_name = row["Model"]

    # Check if station exists
    if station not in filtered_sheet_dict:
        print(f"Station {station} not found in filtered data. Skipping...")
        continue

    df = filtered_sheet_dict[station].copy()

    # Ensure YEAR is treated as a simple integer for proper chronological sorting
    if isinstance(df['YEAR'].iloc[0], str):
        df['YEAR'] = pd.to_numeric(df['YEAR'], errors='coerce')
    
    # Add El Niño indicator and stronger signal for recent ones
    df['is_el_nino'] = df['YEAR'].apply(lambda x: 1 if x in el_nino_years else 0)
    
    # Add more importance to recent El Niño events which were stronger
    df['recent_strong_el_nino'] = df['YEAR'].apply(lambda x: 2 if x in [2019, 2023] else (1 if x == 2013 else 0))
    
    # Apply feature engineering to help with extreme values
    df = engineer_features(df)
    
    # Sort by year to ensure chronological order
    df = df.sort_values('YEAR')
    
    # Print year range for verification
    print(f"\nWorking with {station} data from {df['YEAR'].min()} to {df['YEAR'].max()}")
    print(f"Total data points: {len(df)}")

    # Log feature names for reference
    feature_columns = [col for col in df.columns if col != 'YEAR' and col != df.columns[1]]
    print(f"Features used: {', '.join(feature_columns)}")

    target_col = df.columns[1]  # Assuming this is the rainfall column
    
    # Save years separately before dropping for plotting
    years = df['YEAR'].values
    
    # Prepare features and target
    X = df.drop(columns=["YEAR", target_col])
    y = df[target_col]

    # Skip if no features
    if X.shape[1] == 0:
        print(f"No features available for {station}. Skipping model creation.")
        continue

    # Create model with appropriate hyperparameters for extreme value capture
    if model_name == "Random Forest":
        # Deeper trees and more estimators to capture complex patterns
        model = Pipeline([
            ('scaler', RobustScaler()),  # Robust to outliers
            ('model', RandomForestRegressor(
                n_estimators=200, 
                max_depth=None,  # Let trees grow deeper
                min_samples_leaf=1,  # Allow leaf nodes with just 1 sample
                min_samples_split=2,
                bootstrap=True,
                random_state=42
            ))
        ])
    elif model_name == "Gradient Boosting":
        # Use a huber loss to be more robust to outliers
        model = Pipeline([
            ('scaler', StandardScaler()),
            ('model', GradientBoostingRegressor(
                loss='huber',  # More robust to outliers than squared error
                n_estimators=200,
                learning_rate=0.05,  # Slower learning rate to avoid overfitting
                max_depth=6,
                min_samples_leaf=3,
                subsample=0.8,  # Use 80% of samples in each boosting iteration
                random_state=42
            ))
        ])
    elif model_name == "SVR":
        # Use RBF kernel with balanced C and epsilon parameters
        model = Pipeline([
            ('scaler', StandardScaler()),  # SVR needs scaling
            ('model', SVR(
                kernel='rbf',
                C=10.0,        # Higher C gives more weight to outliers
                epsilon=0.1,   # Smaller epsilon allows more support vectors
                gamma='scale' 
            ))
        ])
    elif model_name == "Neural Network":
        # Deeper network with dropout for regularization
        model = Pipeline([
            ('scaler', StandardScaler()),  # Neural networks need scaling
            ('model', MLPRegressor(
                hidden_layer_sizes=(100, 50, 25),  # Deeper architecture
                activation='relu',
                solver='adam',
                alpha=0.0001,  # L2 regularization parameter
                batch_size='auto',
                learning_rate='adaptive',
                max_iter=2000,
                early_stopping=True,  # Use early stopping to prevent overfitting
                validation_fraction=0.1,
                random_state=42
            ))
        ])
    else:
        print(f"Unknown model type: {model_name}. Skipping...")
        continue

    # Skip if not enough data
    if len(X) < 5:
        print(f"Not enough data points for {station}. Skipping model creation.")
        continue

    # Calculate split indices based on your desired percentages
    train_size = 0.75  # 50% for training
    val_size = 0.15    # 30% for validation
    test_size = 0.1   # 20% for testing

    # Calculate split points
    train_end = int(len(X) * train_size)
    val_end = train_end + int(len(X) * val_size)

    # Print the number of samples in each split
    print(f"Training samples: {train_end}")
    print(f"Validation samples: {val_end - train_end}")
    print(f"Testing samples: {len(X) - val_end}")

    # Split data while preserving time order
    X_train = X.iloc[:train_end]
    y_train = y.iloc[:train_end]
    years_train = years[:train_end]

    X_val = X.iloc[train_end:val_end]
    y_val = y.iloc[train_end:val_end]
    years_val = years[train_end:val_end]

    X_test = X.iloc[val_end:]
    y_test = y.iloc[val_end:]
    years_test = years[val_end:]

    # Print year ranges for verification
    print(f"Training years: {min(years_train)} to {max(years_train)}")
    print(f"Validation years: {min(years_val)} to {max(years_val)}")
    print(f"Testing years: {min(years_test)} to {max(years_test)}")

    try:
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict on validation and test sets
        y_val_pred = model.predict(X_val)
        y_test_pred = model.predict(X_test)
        
        # Evaluate on test set
        mse = mean_squared_error(y_test, y_test_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_test_pred)
        mae = mean_absolute_error(y_test, y_test_pred)
        
        # Special metric for extreme values: Calculate error on values below zero
        below_zero_indices = np.where(y_test < 0)[0]
        if len(below_zero_indices) > 0:
            below_zero_rmse = np.sqrt(mean_squared_error(
                y_test.iloc[below_zero_indices], 
                y_test_pred[below_zero_indices]
            ))
            print(f"  RMSE for values below zero: {below_zero_rmse:.3f}")
            print(f"  Number of below-zero actual values in test set: {len(below_zero_indices)}")
        else:
            below_zero_rmse = None
            print("  No below-zero values in test set to evaluate")
        
        # Store results
        results_dict.setdefault(station, {})[model_name] = {
            'rmse': rmse,
            'r2': r2,
            'mae': mae,
            'below_zero_rmse': below_zero_rmse,
            'predictions': y_test_pred,
            'actuals': y_test.values,
            'years': years_test
        }
        
        print(f"\nResults for {station} using {model_name}:")
        print(f"  R² = {r2:.3f}")
        print(f"  RMSE = {rmse:.3f}")
        print(f"  MAE = {mae:.3f}")
        
        # Print validation metrics
        val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
        print(f"  Validation RMSE = {val_rmse:.3f}")
        
        # Plot with proper x-axis formatting and focus on extremes
        plt.figure(figsize=(14, 8))
        
        # Create a more detailed visualization
        plt.subplot(2, 1, 1)
        plt.plot(years_test, y_test.values, label="Observed", marker='o', color='blue')
        plt.plot(years_test, y_test_pred, label="Predicted", linestyle='--', marker='x', color='red')
        
        # Add a horizontal line at y=0 to highlight the zero threshold
        plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
        
        plt.title(f"{station} - {model_name}: Observed vs Predicted (Test Set)")
        plt.xlabel("Year")
        plt.ylabel(f"{target_col}")
        plt.legend()
        plt.grid(True)
        
        # Ensure x-axis shows actual years
        plt.xticks(years_test)
        if len(years_test) > 6:
            plt.xticks(rotation=45)
        
        # Add a second subplot focusing on error analysis
        plt.subplot(2, 1, 2)
        errors = y_test.values - y_test_pred
        plt.bar(years_test, errors, color='purple', alpha=0.6)
        plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
        plt.title("Prediction Errors (Actual - Predicted)")
        plt.xlabel("Year")
        plt.ylabel("Error")
        plt.grid(True, axis='y')
        
        # Ensure x-axis shows actual years
        plt.xticks(years_test)
        if len(years_test) > 6:
            plt.xticks(rotation=45)
            
        plt.tight_layout()
        plt.show()

        # Create a scatterplot to evaluate how well the model predicts across the range
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test.values, y_test_pred, alpha=0.6)
        
        # Add perfect prediction line
        min_val = min(min(y_test.values), min(y_test_pred))
        max_val = max(max(y_test.values), max(y_test_pred))
        plt.plot([min_val, max_val], [min_val, max_val], 'k--')
        
        # Add vertical line at y=0
        plt.axvline(x=0, color='red', linestyle='--', alpha=0.3)
        # Add horizontal line at y=0
        plt.axhline(y=0, color='red', linestyle='--', alpha=0.3)
        
        plt.title(f"{station} - {model_name}: Actual vs Predicted Values")
        plt.xlabel("Actual Values")
        plt.ylabel("Predicted Values")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"Error training {model_name} for {station}: {str(e)}")

# After running all models, create a summary dataframe of best models
summary_rows = []

for station in results_dict:
    station_models = results_dict[station]
    
    # Find the best model based on R² score
    best_model_name = max(station_models.keys(), key=lambda k: station_models[k]['r2'])
    best_model = station_models[best_model_name]
    
    # Get features used
    features_used = ', '.join(feature_columns)
    
    # Add to summary
    summary_rows.append({
        'Station': station,
        'Best Model': best_model_name,
        'R2': best_model['r2'],
        'RMSE': best_model['rmse'],
        'MAE': best_model['mae'],
        'Features Used': features_used
    })

# Create and display summary dataframe
summary_df = pd.DataFrame(summary_rows)
print("\n" + "="*70)
print("Summary of Best Models for Each Station")
print("="*70)
print(summary_df.to_string(index=True))
print("="*70)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create a dataframe to store the best models and their performance metrics
best_models_summary = pd.DataFrame(columns=[
    'Station', 'Best Model', 'R2', 'RMSE', 'MAE', 'Features Used'
])

# Create a dictionary to store forecasting-ready dataframes for each station
forecasting_dfs = {}

# Process results from cell 4 to identify the best model for each station
print("\n" + "="*70)
print("BEST MODEL SELECTION SUMMARY")
print("="*70)

for station, models_dict in results_dict.items():
    print(f"\nAnalyzing models for station: {station}")
    
    best_model_name = None
    best_r2 = float('-inf')
    best_metrics = {}

    # Find the model with the highest R² value from the results_dict (from cell 4)
    for model_name, metrics in models_dict.items():
        r2 = metrics['r2']
        print(f"  {model_name}: R² = {r2:.4f}, RMSE = {metrics['rmse']:.4f}")
        
        if r2 > best_r2:
            best_r2 = r2
            best_model_name = model_name
            best_metrics = metrics
    
    if best_model_name:
        try:
            print(f"\n✅ Best model for {station}: {best_model_name}")
            print(f"  R² = {best_r2:.4f}")
            print(f"  RMSE = {best_metrics['rmse']:.4f}")
            print(f"  MAE = {best_metrics['mae']:.4f}")
            
            # Get the original dataframe for this station
            station_df = filtered_sheet_dict[station].copy()
            
            # Get the features used for this station - use exactly the same features from cell 4
            features_used = [col for col in station_df.columns if col not in ['YEAR', station]]
            features_str = ', '.join(features_used)
            
            # Add El Niño indicator for future forecasting
            el_nino_years = [1997, 2013, 2019, 2023]
            station_df['is_el_nino'] = station_df['YEAR'].apply(lambda x: 1 if int(x) in el_nino_years else 0)
            
            best_models_summary = pd.concat([
                best_models_summary,
                pd.DataFrame({
                    'Station': [station],
                    'Best Model': [best_model_name],
                    'R2': [best_r2],
                    'RMSE': [best_metrics['rmse']],
                    'MAE': [best_metrics['mae']],
                    'Features Used': [features_str]
                })
            ], ignore_index=True)
            
            # Store the original dataframe with the best model predictions for forecasting
            forecasting_dfs[station] = {
                'df': station_df,
                'model_name': best_model_name,
                'features': features_used,
                'target': station,
                'years': best_metrics['years'],
                'actuals': best_metrics['actuals'],
                'predictions': best_metrics['predictions'],
                'r2': best_r2,
                'rmse': best_metrics['rmse'],
                'mae': best_metrics['mae']
            }

            
            years_test = best_metrics['years']
            y_test = best_metrics['actuals']
            y_pred = best_metrics['predictions']
            target_col = station

            print("Years in test set:", years_test)

            if len(set(years_test)) == 1:
                print("WARNING: All test data is from the same year!")

            plt.figure(figsize=(12, 5))

            if len(set(years_test)) == 1:
                x_vals = np.arange(len(y_test))
                plt.plot(x_vals, y_test, label="Observed", marker='o')
                plt.plot(x_vals, y_pred, label="Predicted", linestyle='--', marker='x')
                plt.xticks(x_vals, [years_test[0]] * len(y_test))
            else:
                plt.plot(years_test, y_test, label="Observed", marker='o')
                plt.plot(years_test, y_pred, label="Predicted", linestyle='--', marker='x')

            plt.title(f"{station} - {best_model_name}: Observed vs Predicted (Test Set)")
            plt.xlabel("Year")
            plt.ylabel(f"{target_col}")
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.show()

        except Exception as e:
            print(f"Error handling best model for {station}: {str(e)}")
            
print("\n" + "="*70)
print("BEST MODELS SUMMARY")
print("="*70)
display(best_models_summary)

print("\nPrepared forecasting dataframes for these stations:")
for station in forecasting_dfs.keys():
    print(f"- {station}")

print("\nForecasting-ready dataframes and best models are stored in 'forecasting_dfs'")
print("You can use these in cell 6 for drought prediction and forecasting.")


In [ ]:
print(forecasting_dfs) 

In [ ]:

from statsmodels.tsa.arima.model import ARIMA
import traceback

# Define drought threshold
DROUGHT_THRESHOLD = -0.5

# Dictionary to store results for all stations
drought_analysis = {}

# First, make sure we have the forecasting dataframes
# This assumes you have already created a forecasting_dfs dictionary from your model results
# If not, let's create it based on your best models

# Get the list of stations from the results_dict
stations = list(results_dict.keys())

# Create a base forecasting dataframe with years
# Assuming we want to analyze the test period data first
forecasting_df = pd.DataFrame()
forecasting_df['YEAR'] = np.array([])  # Will be populated below

# Now let's analyze each station
for station in stations:
    print("="*70)
    print(f"{station} STATION DROUGHT ANALYSIS")
    print("="*70)
    
    # Get the best model results for this station
    best_model_name = max(results_dict[station].keys(), key=lambda k: results_dict[station][k]['r2'])
    station_results = results_dict[station][best_model_name]
    
    # Get years and actual rainfall values from test set
    years = station_results['years']
    rainfall = station_results['actuals']  # Actual values
    
    # Add to forecasting dataframe if it's empty
    if len(forecasting_df['YEAR']) == 0:
        forecasting_df['YEAR'] = years
    
    # Add station data to forecasting dataframe
    forecasting_df[station] = rainfall
    
    # Create container for this station's analysis
    drought_analysis[station] = {}

    # Step 1: Identify drought years
    drought_years = years[rainfall <= DROUGHT_THRESHOLD]
    drought_frequency = len(drought_years) / len(years) * 100

    print(f"\nHistorical drought years: {drought_years}")
    print(f"Historical drought frequency: {drought_frequency:.1f}%")

    # Store results
    drought_analysis[station]['historical_drought_years'] = drought_years.tolist() if hasattr(drought_years, 'tolist') else list(drought_years)
    drought_analysis[station]['historical_frequency'] = drought_frequency

    # Step 2: Compute average drought interval if applicable
    if len(drought_years) > 1:
        intervals = np.diff(drought_years)
        avg_interval = np.mean(intervals)
        print(f"\nAverage interval between droughts: {avg_interval:.1f} years")
        drought_analysis[station]['avg_interval'] = float(avg_interval)
    else:
        avg_interval = None
        print("\nNot enough drought events to compute average interval.")
        drought_analysis[station]['avg_interval'] = None

    # Step 3: Forecast future droughts using ARIMA
    try:
        # Prepare time series
        # We need a longer series for ARIMA, so let's use all available data
        # For demonstration, I'll just use the test set data but in practice you should use your full historical dataset
        ts_data = pd.Series(rainfall, index=years)
        ts_data.name = 'Rainfall'
        ts_data = ts_data.dropna()

        # Check if we have enough data points
        if len(ts_data) < 8:  # Lowered from 20 to 8 for demonstration
            raise ValueError(f"Not enough non-NaN data points for ARIMA (only {len(ts_data)} available).")

        # Fit ARIMA model (p=1, d=1, q=1)
        # In production, you might want to use auto_arima to find the best parameters
        model = ARIMA(ts_data, order=(1, 1, 0))  # Simplified to (1,1,0) for better stability
        model_fit = model.fit()

        # Forecast next 10 years
        forecast_years = np.arange(ts_data.index[-1] + 1, ts_data.index[-1] + 11)
        forecast = model_fit.forecast(steps=10)

        # Identify predicted drought years
        forecast_drought_flags = forecast <= DROUGHT_THRESHOLD
        predicted_drought_years = forecast_years[forecast_drought_flags]

        print("\nPredicted rainfall values for the next 10 years:")
        for year, value in zip(forecast_years, forecast):
            drought_status = "DROUGHT PREDICTED" if value <= DROUGHT_THRESHOLD else "No drought"
            print(f"{year}: {value:.3f} ({drought_status})")

        # Store forecast results
        drought_analysis[station]['forecast_years'] = forecast_years.tolist() if hasattr(forecast_years, 'tolist') else list(forecast_years)
        drought_analysis[station]['forecast_values'] = forecast.tolist() if hasattr(forecast, 'tolist') else list(forecast)
        drought_analysis[station]['predicted_drought_years'] = predicted_drought_years.tolist() if hasattr(predicted_drought_years, 'tolist') else list(predicted_drought_years)

        if len(predicted_drought_years) > 0:
            next_drought = predicted_drought_years[0]
            print(f"\nNext predicted drought for {station}: {next_drought}")
            years_until_drought = next_drought - ts_data.index[-1]
            print(f"Years until next drought: {years_until_drought}")
            drought_analysis[station]['next_drought_year'] = int(next_drought)
            drought_analysis[station]['years_until_drought'] = int(years_until_drought)
        else:
            print(f"\nNo droughts predicted for {station} in the next 10 years")
            drought_analysis[station]['next_drought_year'] = None
            drought_analysis[station]['years_until_drought'] = None

    except Exception as e:
        print(f"\nError in ARIMA forecast for {station}: {str(e)}")
        print(traceback.format_exc())
        print("Falling back to pattern-based estimation...")

        # Fallback to historical pattern if possible
        if avg_interval is not None and len(drought_years) > 0:
            last_drought = drought_years[-1]
            pattern_next_drought = int(last_drought + avg_interval)
            print(f"Based on historical patterns (avg interval: {avg_interval:.1f} years):")
            print(f"Next expected drought around: {pattern_next_drought}")
            drought_analysis[station]['pattern_next_drought'] = pattern_next_drought
        else:
            print("No pattern-based prediction available due to insufficient data.")
            drought_analysis[station]['pattern_next_drought'] = None

# Create visualizations for drought analysis
import matplotlib.pyplot as plt

# Plot historical droughts and predictions for each station
plt.figure(figsize=(15, 10))

for i, station in enumerate(stations):
    plt.subplot(len(stations), 1, i+1)
    
    # Get results for this station
    results = drought_analysis[station]
    
    # Plot historical data points
    years = forecasting_df['YEAR']
    values = forecasting_df[station]
    
    # Plot the full rainfall line
    plt.plot(years, values, 'b-', label='Historical')
    
    # Highlight drought points
    drought_mask = values <= DROUGHT_THRESHOLD
    if any(drought_mask):
        plt.scatter(years[drought_mask], values[drought_mask], color='red', s=50, label='Historical Droughts')
    
    # Plot forecast if available
    if 'forecast_years' in results and 'forecast_values' in results:
        forecast_years = results['forecast_years']
        forecast_values = results['forecast_values']
        plt.plot(forecast_years, forecast_values, 'g--', label='ARIMA Forecast')
        
        # Highlight predicted droughts
        forecast_drought_mask = np.array(forecast_values) <= DROUGHT_THRESHOLD
        if any(forecast_drought_mask):
            plt.scatter(
                np.array(forecast_years)[forecast_drought_mask], 
                np.array(forecast_values)[forecast_drought_mask], 
                color='red', marker='x', s=50, label='Predicted Droughts'
            )
    
    # Add pattern-based prediction if available
    if results.get('pattern_next_drought') is not None:
        pattern_year = results['pattern_next_drought']
        plt.axvline(x=pattern_year, color='purple', linestyle='--', 
                    label=f'Pattern-based ({pattern_year})')
    
    # Add drought threshold line
    plt.axhline(y=DROUGHT_THRESHOLD, color='r', linestyle='-', alpha=0.3, 
                label=f'Drought Threshold ({DROUGHT_THRESHOLD})')
    
    plt.title(f"{station} - Drought Analysis")
    plt.xlabel("Year")
    plt.ylabel("Rainfall Anomaly")
    plt.grid(True)
    plt.legend(loc='best')

plt.tight_layout()
plt.show()

# Summary of Drought Predictions
print("\n" + "="*70)
print("SUMMARY OF DROUGHT PREDICTIONS")
print("="*70)

summary_data = []
for station, results in drought_analysis.items():
    print(f"\n{station}:")
    print(f"  Historical drought frequency: {results['historical_frequency']:.1f}%")

    next_drought = "None predicted"
    method = "N/A"
    
    if results.get('next_drought_year') is not None:
        next_drought = results['next_drought_year']
        years_until = results['years_until_drought']
        print(f"  Next drought (ARIMA-based): {next_drought} ({years_until} years from last data point)")
        method = "ARIMA"
    elif results.get('pattern_next_drought') is not None:
        next_drought = results['pattern_next_drought']
        print(f"  Next drought (Pattern-based): {next_drought}")
        method = "Pattern"
    else:
        print(f"  No drought prediction available")
    
    # Add to summary data
    summary_data.append({
        'Station': station,
        'Historical Frequency (%)': results['historical_frequency'],
        'Next Drought Year': next_drought,
        'Prediction Method': method
    })

# Create summary dataframe
summary_df = pd.DataFrame(summary_data)
print("\nDrought Prediction Summary Table:")
print(summary_df.to_string(index=False))
